<a href="https://colab.research.google.com/github/LeoDardanello/DA_Semantic_Segmentation/blob/main/Semantic_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
import numpy